In [ ]:
%autosave 0

In [ ]:
from __future__ import print_function, absolute_import, division

import logging
import os
import sys
from collections import defaultdict, OrderedDict
from errno import ENOENT, ENODATA
from stat import S_IFDIR, S_IFLNK, S_IFREG
from sys import argv, exit, path
import time
from binascii import hexlify

if not hasattr(__builtins__, 'bytes'):
    bytes = str

In [ ]:
from datetime import datetime

In [ ]:
sys.path.insert(0,"../si446x/")
sys.path.insert(0,"../tagfuse")
#from tagfuse import *

#from tagfuse.radioutils import radio_start
#radio = radio_start()

In [ ]:
from si446x import Si446xRadio
radio = Si446xRadio(0)
radio.unshutdown()

In [ ]:
from si446x import Si446xRadio
radio=Si446xRadio(0)
if (radio == None):
    raise RuntimeError('radio_start: could not instantiate radio')
radio.unshutdown()
radio.write_config()
radio.config_frr()

In [ ]:
from si446x import get_config_wds, get_name_wds
get_name_wds()

In [ ]:
radio.trace.display(radio.trace.filter())

In [ ]:
radio.get_clear_interrupts()

In [ ]:
#for i in range(100):
#    print(radio.get_clear_interrupts())

In [ ]:
radio.dump_radio()

In [ ]:
last_dump = {'\x00': bytearray(b'R\x00\x18`\x00\x00\x01`\x00\x00\x00'),
 '\x01': bytearray(b'\x07;#('),
 '\x02': bytearray(b'\x01\x02\t\x00'),
 '\x10': bytearray(b'\x08\x14\x00\x0f1\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
 '\x11': bytearray(b'\x01\xb4+\x00\x00\x00\x00\x00\x00\x00'),
 '\x12': bytearray(b'\x85\x01\x08\xff\xff\x00\x82\x00*\x01\x00\x10\x10\x00\x01\x04\xa2\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x04\x82\x00\xff\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
 ' ': bytearray(b'\x03\x00\x07\x06\x1a\x80\x05\xc9\xc3\x80\x00\x05v\x00\x00g`M6!\x11\x08\x03\x01\x01\x80\x08\x03\x80\x00  \x00\xe8\x01w\x01]\x86\x00\xaf\x02\xc0\x00\x94#\x81U\x01]\x80\x00\x00\xe0\x00\x00\x11RR\x80\x1a\xff\xff\x00*\x0c\x84#V\x8f\x00\xde\x01\x00 \x0c"\x1a@\x03\x84\n\x00(\x83A\xff\x00\x00\x025fd'),
 '!': bytearray(b'\xa2\x81&\xaf?\xee\xc8\xc7\xdb\xf2\x02\x08\x07\x03\x15\xfc\x0f\x00\xa2\x81&\xaf?\xee\xc8\xc7\xdb\xf2\x02\x08\x07\x03\x15\xfc\x0f\x00'),
 '"': bytearray(b'\x085\x00\x1d\x80#\x03'),
 '#': bytearray(b',\x0e\x0b\x04\x0cs\x03\x05'),
 '0': bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
 '@': bytearray(b'8\r\xdd\xdd\x00\x0e \xfe'),
 'P': bytearray(b'\x04\x01\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?')}

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from si446xdef import *


def si446x_device_group_fetch_and_decode(group):
    gname = radio_config_group_ids.decoding[group]
    g_s = radio_config_groups[radio_config_group_ids.build(gname)]
    gid = radio_config_group_ids.build(gname)
    p = si446x_device_get_property(radio, gname, 0, g_s.sizeof())
    print(g_s, insert_space(p))
    #print(gname, len(p), hexlify(p))
    print(radio_display_structs[g_s](g_s, p))
    return None


def si446x_device_command_fetch_and_decode(cmd):
    cname = radio_status_cmd_ids.decoding[cmd]
    cfunc, cstr = radio_status_commands[radio_config_cmd_ids.build(cname)]
#    print(cname, cmd)
    if (cfunc):
        cmd = cfunc(cname)
        if (cmd):
#            print(cfunc, hexlify(cmd), cstr)
            radio.spi.command(cmd, cstr)
            rsp = radio.spi.response(cstr.sizeof(), cstr.name)
            if (rsp):
#                print(cstr, radio_display_structs[cstr])
                print(cstr, insert_space(rsp))
                print(radio_display_structs[cstr](cstr, rsp))
            else:
                print('no response')
        else:
            print('no command')
    else:
        print('no function')
    return None

def si446x_device_get_property(radio, g_n, start, limit):
    prop_x = 0
    prop_b = bytearray()
    while (prop_x < limit):
        chunk_size = limit - prop_x
        x = MAX_RADIO_RSP if (chunk_size >= MAX_RADIO_RSP) else chunk_size
        rsp = radio.get_property(g_n, prop_x, x)
        if (rsp):
            prop_b += bytearray(rsp[:x])
            prop_x += x
        else:
            return None
    return prop_b

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

## Compare configurations

In [ ]:
STOP

In [ ]:
get_config_wds(10)

In [ ]:
rpi = radio.dump_radio()
rpi

In [ ]:
tagold = {'!': bytearray(b'\xa2\x81&\xaf?\xee\xc8\xc7\xdb\xf2\x02\x08\x07\x03\x15\xfc\x0f\x00\xa2\x81&\xaf?\xee\xc8\xc7\xdb\xf2\x02\x08\x07\x03\x15\xfc\x0f\x00'),
       '@': bytearray(b'8\r\xdd\xdd\x00\x0e \xfe'),
       '#': bytearray(b',\x0e\x0b\x04\x0cs\x03\x05'),
       '"': bytearray(b'\x08\x10\x00\x1d\x80#\x03'),
       '\x10': bytearray(b'\x08\x14\x00\x0f1\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
       ' ': bytearray(b'\x03\x00\x07\x06\x1a\x80\x05\xc9\xc3\x80\x00\x05v\x00\x00g`M6!\x11\x08\x03\x01\x01\x80\x08\x03\x80\x00  \x00\xe8\x01w\x01]\x86\x00\xaf\x02\xc0\x00\x94#\x81U\x01]\x80\x00\x00\xe0\x00\x00\x11RR\x80\x1a\xff\xff\x00*\x0c\x84#V\x8f\x00\xde\x01\x00 \x0c"\x1a@\x03\x84\n\x00(\x83A\xff\x00\x00\x025fd\x07x"'),
       '\x00': bytearray(b'R\x00\x18`\x00\x00\x01`\x00\x00\x00'),
       '0': bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
       '\x11': bytearray(b'\x01\xb4+\x00\x00\x00\x00\x00\x00\x00'),
       'P': bytearray(b'\x04\x01\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?'),
       '\x01': bytearray(b'\x07;#('),
       '\x12': bytearray(b'\x85\x01\x08\xff\xff\x00\x82\x00*\x01\x00(\x19\x00\x01\x04\xa2\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x04\x82\x00\xff\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
       '\x02': bytearray(b'\t\x04\x06\n')}
tagold

In [ ]:
tag = {'!': bytearray(b'\xa2\x81&\xaf?\xee\xc8\xc7\xdb\xf2\x02\x08\x07\x03\x15\xfc\x0f\x00\xa2\x81&\xaf?\xee\xc8\xc7\xdb\xf2\x02\x08\x07\x03\x15\xfc\x0f\x00'), '@': bytearray(b'8\r\xdd\xdd6\x9d \xfe'), '#': bytearray(b',\x0e\x0b\x04\x0cs\x03\x05'), '"': bytearray(b'\x08\x10\x00\x1d\x80#\x03'), '\x10': bytearray(b'\x08\x14\x00\x0f1\x00\x00\x00\x00\x00\x00\x00\x00\x00'), ' ': bytearray(b'\x03\x00\x07\x06\x1a\x80\x05\xc9\xc3\x80\x00\x05v\x00\x00g`M6!\x11\x08\x03\x01\x01\x80\x08\x03\x80\x00  \x00\xe8\x01w\x01]\x86\x00\xaf\x02\xc2\x00\x04#\x80\x1d\x10\x04\x80\x00\x00\xe0\x00\x00\x11RR\x80\x1a\xff\xff\x00*\x0c\xa4"V\x83\x00\xde\x01\x00 \x0c"\x18@\x03\x84\n\x00(\x03\x07\xff\x00\x00\x00\x00@\x04\x07x '), '\x00': bytearray(b'R\x00\x18`\x00\x00\x01`\x00\x00\x00'), '0': bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), '\x11': bytearray(b'\x01\xb4+\x00\x00\x00\x00\x00\x00\x00'), 'P': bytearray(b'\x04\x01\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?'), '\x01': bytearray(b'\x07;#('), '\x12': bytearray(b'\x85\x01\x08\xff\xff\x00\x82\x00*\x01\x00(\x19\x00\x01\x04\xa2\x00\x00\x00"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x04\x82\x00\xff\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'), '\x02': bytearray(b'\t\x04\x06\n')}
tag

In [ ]:
for key in sorted(tag.keys()):
    if tag[key] != rpi[key]:
        print(hex(ord(key)))
        st = tag[key]
        sr = rpi[key]
        print(hexlify(st))
        print(hexlify(sr))
        ss = ''
        for x in range(min(len(st),len(sr))):
            if st[x] == sr[x]:
                ss += ' '
            else:
                ss += '^'
        print(ss)
        for x in range(min(len(st),len(sr))):
            if st[x] != sr[x]:
                print(x, hex(st[x]), hex(sr[x]))

In [ ]:
handler.radio.trace.display(handler.radio.trace.filter())

In [ ]:
radio.config_frr()

## Change configuration

In [ ]:
import sys
sys.path.insert(0,"../si446x/si446x/radioconfig")

In [ ]:
import si446xcfg
help(si446xcfg)

In [ ]:
si446xcfg.get_config_device(24)

In [ ]:
si446xcfg.get_config_wds(0)

In [ ]:
si446xcfg.get_name_wds()

In [ ]:
si446xcfg.wds_config_count()

In [ ]:
len(si446xcfg.wds_config_str()[0])

In [ ]:
si446xcfg.wds_default_config(1)

In [ ]:
si446xcfg.wds_default_config()

In [ ]:
for i in range(5):
    if (si446xcfg.wds_default_config(i) < i):
        break
    print(si446xcfg.get_name_wds())    

In [ ]:
x=0
count=0
while (True):
    s=bytearray(si446xcfg.get_config_wds(x))
    print(hexlify(s))
    if len(s) == 0: break
    print(x, len(s))
    x += len(s) + 1
    count +=1
count, x

In [ ]:
x=0
count=0
while (True):
    s=bytearray(si446xcfg.get_config_device(x))
    print(hexlify(s))
    if len(s) == 0: break
    print(x, len(s))
    x += len(s) + 1
    count +=1
count, x

In [ ]:
reload(si446xcfg)

In [ ]:
type(s), len(s)

In [ ]:
x=0

In [ ]:
bb = bytearray(si446xcfg.get_config_wds(x))
x += len(bb) + 1
x, len(bb), bb

In [ ]:
si446xcfg.get_name_wds()

In [ ]:
from binascii import hexlify

In [ ]:
radio.config_strings